#Fine-Tuning GPT-3

Copyright 2023 Denis Rothman

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)

Check the cost of fine-tuning your dataset on OpenAI before running the notebook.

Run this notebook cell by cell to:

1.prepare data
2.fine-tune a model
3.run a fine-tuned model
4.manage the fine-tunes

## Installing OpenAI & Wandb

Restart the runtime after installing openai and run the cell again to make sur that "import openai" is executed.

In [ ]:
try:
  import openai
except:
  !pip install openai
  import openai

## Your API Key

In [2]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)

#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [5]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
try:
  import wandb
except:
  !pip install wandb
  import wandb

# 1.Preparing the dataset

## 1.1. Preparing the data in JSON

In [ ]:
#From Gutenberg to JSON
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import requests
from bs4 import BeautifulSoup
import json
import re

# First, fetch the text of the book from Project Gutenberg
url = 'http://www.gutenberg.org/cache/epub/4280/pg4280.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Get the text of the book and clean it up a bit
text = soup.get_text()
text = re.sub('\s+', ' ', text).strip()

# Split the text into sentences
sentences = sent_tokenize(text)

# Define the separator and ending
prompt_separator = " ->"
completion_ending = "\n"

# Now create the prompts and completions
data = []
for i in range(len(sentences) - 1):
    data.append({
        "prompt": sentences[i] + prompt_separator,
        "completion": " " + sentences[i + 1] + completion_ending
    })

# Write the prompts and completions to a file
with open('kant_prompts_and_completions.json', 'w') as f:
    for line in data:
        f.write(json.dumps(line) + '\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd

# Load the data
df = pd.read_json('kant_prompts_and_completions.json', lines=True)
df

,prompt,completion
0,The Project Gutenberg Etext of The Critique of...,Be sure to check the copyright laws for your ...
1,Be sure to check the copyright laws for your c...,"We encourage you to keep this file, exactly a..."
2,"We encourage you to keep this file, exactly as...",Please do not remove this.\n
3,Please do not remove this. ->,This header should be the first thing seen wh...
4,This header should be the first thing seen whe...,Do not change or edit it without written perm...
...,...,...
6122,"78-79. is their motto, under which they may le...",As regards those who wish to pursue a scienti...
6123,As regards those who wish to pursue a scientif...,"When I mention, in relation to the former, th..."
6124,"When I mention, in relation to the former, the...",The critical path alone is still open.\n
6125,The critical path alone is still open. ->,If my reader has been kind and patient enough...


##  1.2. Converting the data to JSONL

Answer "Y" to all of the questions.

In [ ]:
!openai tools fine_tunes.prepare_data -f "kant_prompts_and_completions.json"

In [ ]:
import json

# Open the file and read the lines
with open('kant_prompts_and_completions_prepared.jsonl', 'r') as f:
    lines = f.readlines()

# Parse and print the first 5 lines
for line in lines[199:300]:
    data = json.loads(line)
    print(json.dumps(data, indent=4))

# 2.Fine-tuning a model

In [ ]:
!openai api fine_tunes.create -t "kant_prompts_and_completions_prepared.jsonl" -m "ada"

OpenAI has many requests.
If your steam is interrupted, OpenAI will indicate the instruction to continue fine-tuning.

In [ ]:
# Uncomment this cell to activate the fine_tunes 'follow' instruction
#!openai api fine_tunes.follow -i [YOUR_FINE_TUNE]

# 3.Running the fine-tuned GPT-3 model

We will now run the model for a completion task

Note: If your fine-tuned model does not appear immediately after the end of the fine-tuning process, you might have to wait until it is processed by OpenAI. You can also:

1.go to the OpenAI Playground to test your model: https://platform.openai.com/playground

2.select your model in the dropdown list and test it in that environment

In [ ]:
f = open("drive/MyDrive/files/fine_tune.txt", "r")
FINE_TUNE=f.readline().strip() #load your saved model from a file or load it in this variable
f.close()
FINE_TUNE

In [6]:
prompt = "Freedom can be a concept or a virtue ->"
response=openai.Completion.create(
  model=FINE_TUNE, #Your model in FINE_TUNE,
  prompt=prompt,
  temperature=1,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop="\n",
  max_tokens=200
)

In [1]:
response

In [8]:
import textwrap
generated_text = response['choices'][0]['text']

# Remove leading and trailing whitespace
generated_text = generated_text.strip()

# Convert to a pretty paragraph by replacing newline characters with spaces
single_line_response = generated_text.replace('\n', ' ')

# Use textwrap.fill to nicely format the paragraph to wrap at 80 characters (or whatever width you prefer)
wrapped_response = textwrap.fill(single_line_response, width=80)
print(wrapped_response)

At first glance, solely through our language, none of these conceptions better
than their opposites—that is, in an empirical intuition.


# 4.Managing the fine_tunes

In [ ]:
# List all created fine-tunes
!openai api fine_tunes.list > fine_tunes.json
!openai api fine_tunes.list

**ChatGPT PLUS, GPT-4 provides a  breakdown of the components of the JSON object**

- `"object"`: This line specifies the type of object the JSON is representing. Here it's a fine-tuned model.

- `"id"`: This is the unique identifier for this fine-tuning job. This ID is typically used to reference this specific instance of fine-tuning.

- `"hyperparams"`: These are the hyperparameters used for fine-tuning the model.
   - `"n_epochs"`: Number of epochs for the training, i.e., how many times the learning algorithm will work through the entire training dataset.
   - `"batch_size"`: The number of training examples used in one iteration (or update) of model parameters.
   - `"prompt_loss_weight"`: This is the weight assigned to the loss function of the prompts during training. A higher value places more emphasis on minimizing the loss of the prompts.
   - `"learning_rate_multiplier"`: This value is used to scale the learning rate during training. A lower value will cause the model to learn slower and vice versa.

- `"organization_id"`: This is the identifier for the organization account under which the fine-tuning operation was performed.

- `"model"`: The base model used for fine-tuning. In your case, it's `ada`, which is a version of GPT-3.

- `"training_files"`: This array contains information about the files used for training.
  - `"object"`: Specifies the object type, in this case, a file.
  - `"id"`: The unique identifier for this file.
  - `"purpose"`: The purpose of the file, here it's for fine-tuning.
  - `"filename"`: The name of the file.
  - `"bytes"`: The size of the file in bytes.
  - `"created_at"`: The UNIX timestamp for when the file was created.
  - `"status"`: The status of the file processing. Here it's processed.
  - `"status_details"`: Any extra details about the file's status. It's null here, meaning there are no extra details.

- `"validation_files"`: This would include similar details as `"training_files"`, but for any files used for validation during training. It's empty in your case.

- `"result_files"`: This is an array of files that store the result of the fine-tuning operation. The details of each file are similar to those in `"training_files"`.

- `"created_at"`: The UNIX timestamp indicating when this fine-tuning job was created.

- `"updated_at"`: The UNIX timestamp indicating the last time this fine-tuning job was updated.

- `"status"`: The status of the fine-tuning job. In this case, it has succeeded.

- `"fine_tuned_model"`: This is the unique identifier/name for the fine-tuned model.
  
Remember that a UNIX timestamp is the number of seconds that have passed since 00:00:00 Thursday, 1 January 1970, minus leap seconds. Programs like Python's datetime library can convert these to more human-readable formats.

In [ ]:
response = openai.FineTune.retrieve("ft-daprSZy6dWb7KlN6WQxOeS0Y")
print(response)

In [ ]:
import pandas as pd
import json
from datetime import datetime

# Load data from json file
with open('fine_tunes.json') as f:
    data = json.load(f)

# Convert to Pandas DataFrame:
df = pd.json_normalize(data['data'])

# Select specific columns
selected_columns = ['object', 'id', 'fine_tuned_model','status', 'created_at', 'updated_at']
df = df[selected_columns]

# Rename columns for display
column_mapping = {
    'object': 'Object',
    'id': 'ID',
    'fine_tuned_model': 'Fine_Tuned_Model',
    'filename':'Filename',
    'status': 'Status',
    'created_at': 'Created_At',
    'updated_at': 'Updated_At',
}
df.rename(columns=column_mapping, inplace=True)

# Convert UNIX timestamp to standard format
df['Created_At'] = pd.to_datetime(df['Created_At'], unit='s')
df['Updated_At'] = pd.to_datetime(df['Updated_At'], unit='s')

df

In [ ]:
#delete a model
# enter a model in the list of fine-tuned models
#FINE_TUNED_MODEL=[MODEL in list]
#try:
#  openai.Model.delete(FINE_TUNED_MODEL)
#except:
#  print("FINE_TUNED_MODEL not found")